<a href="https://colab.research.google.com/github/Andre2503/Group8-Project4/blob/edrin/FakeNewsCNNV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# INSTALL GRADIO TO RUN THE MODEL WITH A UI

# !pip install -q gradio
!pip install --upgrade gradio

# Also gradio will need the following packages

# !pip install kaleido
# !pip install tiktoken
# !pip install cohere
# !pip install openai



In [10]:
# Import necessary libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense


In [11]:
from google.colab import drive
drive.mount('/content/FakeNews_Processed_Data.csv')


Mounted at /content/FakeNews_Processed_Data.csv


In [12]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Loading the Data


In [13]:
# Loading the Data
import pandas as pd

# the data can be loading from any of the following paths

# file_path = '/content/drive/My Drive/driveAndrea/Resources/FakeNews_Processed_Data.csv'
file_path = '/content/drive/My Drive/Resources/FakeNews_Processed_Data.csv'
df = pd.read_csv(file_path)
print(df.head())




                                          title_text  label
0  law enforcement high alert following threats c...      1
1                         post votes hillary already      1
2  unbelievable obamas attorney general says char...      1
3  bobby jindal raised hindu uses story christian...      0
4  satan russia unvelis image terrifying new supe...      1


# Understanding the Data

In [14]:
# Look at the first few rows of the DataFrame
print(df.head())




                                          title_text  label
0  law enforcement high alert following threats c...      1
1                         post votes hillary already      1
2  unbelievable obamas attorney general says char...      1
3  bobby jindal raised hindu uses story christian...      0
4  satan russia unvelis image terrifying new supe...      1


In [16]:
# Get basic information about the DataFrame
print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title_text  72133 non-null  object
 1   label       72134 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB
None


In [17]:
# Summary statistics for numeric columns
print(df.describe())

              label
count  72134.000000
mean       0.514404
std        0.499796
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000


In [18]:
# i'll get rid of the missing value
df = df.dropna(subset=['title_text'])


# Preprocess The Data
Text cleaning
Tokenization
Sequencing
Padding

In [20]:
# text processing
texts = df['title_text'].astype(str)
labels = df['label'].values

# Initialise and fit the tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad the sequence
max_sequence_length = 100
data = pad_sequences(sequences, maxlen=max_sequence_length)


# Spliting the Data
Divivded data into clening and testing

In [21]:
# Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


# Build the CNN Model
Construct a CNN model suitable for text classification:

In [22]:
# Build the CNN Model
model = Sequential()
model.add(Embedding(10000, 128, input_length=max_sequence_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1280000   
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           82048     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1363349 (5.20 MB)
Trainable params: 1363349 (5.20 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [23]:
# Training the Model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1)



Epoch 1/10
1623/1623 [==============================] - 125s 76ms/step - loss: 0.2251 - accuracy: 0.9046 - val_loss: 0.1467 - val_accuracy: 0.9432
Epoch 2/10
1623/1623 [==============================] - 120s 74ms/step - loss: 0.0670 - accuracy: 0.9769 - val_loss: 0.1436 - val_accuracy: 0.9471
Epoch 3/10
1623/1623 [==============================] - 121s 75ms/step - loss: 0.0107 - accuracy: 0.9975 - val_loss: 0.1742 - val_accuracy: 0.9461
Epoch 4/10
1623/1623 [==============================] - 121s 74ms/step - loss: 0.0037 - accuracy: 0.9996 - val_loss: 0.2140 - val_accuracy: 0.9466
Epoch 5/10
1623/1623 [==============================] - 120s 74ms/step - loss: 0.0038 - accuracy: 0.9992 - val_loss: 0.2406 - val_accuracy: 0.9456
Epoch 6/10
1623/1623 [==============================] - 123s 76ms/step - loss: 0.0093 - accuracy: 0.9969 - val_loss: 0.2302 - val_accuracy: 0.9446
Epoch 7/10
1623/1623 [==============================] - 119s 73ms/step - loss: 0.0057 - accuracy: 0.9981 - val_loss: 0

In [24]:
# prompt: # Evaluate the Model


acc = model.evaluate(X_test, y_test)[1]
print('Accuracy:', acc)


451/451 [==============================] - 7s 15ms/step - loss: 0.3955 - accuracy: 0.9443
Accuracy: 0.9442711472511292


In [25]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


451/451 [==============================] - 6s 13ms/step - loss: 0.3955 - accuracy: 0.9443
Test Accuracy: 0.9442711472511292


In [32]:
# Chat GPT suggested using this to test the model to predict the likelihood that new text is fake news:

news_texts = ["Is South Africa Limiting Water for White People? Proposal Gets Pushback"]

seqs = tokenizer.texts_to_sequences(news_texts)
padded_seqs = pad_sequences(seqs, maxlen=max_sequence_length)
predictions = model.predict(padded_seqs)
print(predictions)  # Closer to 1 indicates fake, closer to 0 indicates real



1/1 [==============================] - 0s 27ms/step
[[0.48055273]]


In [ ]:
# Saving the model
model.save('my_CNNmodel.h5')

#Saving copy to Google drive
model.save('/content/drive/My Drive/path_to_my_model.h5', save_format='h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.save('/content/drive/My Drive/driveAndrea/Resources/CNNmodel.keras')
#model = load_model('my_model.keras')

# UI CODE

Before you run the following code (cells) you must install gradio package and all packages it requires.

Please go to the first cell in this notebook and install one by one all the packages to run the UI.

Thank you.

In [31]:
# THE FOLLOWING CODE IS FOR THE UI
import gradio as gr
import re

nltk.download("stopwords")
nltk.download("punkt")


# the news_analyzer function is the main function of the app.
# it takes the news tittle and body as input and return the
# result as output.
def news_analyzer(news_tittle, news_body):
    ok_data = input_validation(news_tittle, news_body)
    if ok_data == 1:
       # get the data ready for the model
       transformed_data = data_normalization(news_tittle, news_body)
       # get the prediction answer from the model
       model_answer = prediction_answer(transformed_data)
       float_answer = model_answer[0][0]
       # set the answer to be readable
       answer_translate = transformed_answer(float_answer)
       answer = f"The news is {answer_translate} "
    else:
        answer = "Please fill the form with valid data"

    return answer


# the data_normalization function clean and normalize the data as
# our model requires.
def data_normalization(news_title, news_body):
    full_news = news_title + news_body
    full_news = lower_case(full_news)
    clean_text = delete_special_characters(full_news)
    significant_words = delete_stop_words(clean_text)
    standardized_data = significant_words
    # print("data_normalization")
    return standardized_data


# the delete_special_characters function delete all the
# special characters in the text.
def delete_special_characters(text):
    special_characters_deleted = re.sub(r"[^\w\s]|[\d]", "", text)
    # print("delete_special_characters")
    return special_characters_deleted


# the delete_stop_words function delete all the insignificant words (stop words)
# The default list of English stop words includes:
# a, an, and, are, as, at, be, but, by, for, if, in, into, is, it, no, not, of, on, or, such,
# that, the, their, then, there, these, they, this, to, was, will and with.
def delete_stop_words(text):
    # create a list
    list_significant_words = []
    # stop_words get a list of common English stop words provided by nltk
    stop_words = set(stopwords.words("english"))
    # word_tokens is a list of all words in the text.
    # word_tokenize is a function provided by nltk
    # to split the text into words.
    word_tokens = word_tokenize(text)
    # significant_word is a variable of  non-stop word in the text.
    significant_word = [word for word in word_tokens if not word in stop_words]
    # list_significant_words is a string of all significant words in the text.
    list_significant_words = " ".join(significant_word)
    # print("delete_stop_words")
    return list_significant_words

# change whole text to lower case
def lower_case(text):
    lower_case_text = text.lower()
    return lower_case_text

# return the answer from the model
def prediction_answer(data_transformed):
    # load the fake news detector model (fnd_model)

    # new_texts = ["This is a test"]
    new_texts = [data_transformed]

    seqs = tokenizer.texts_to_sequences(new_texts)
    padded_seqs = pad_sequences(seqs, maxlen=max_sequence_length)
    answer_prediction_model = model.predict(padded_seqs)
    return answer_prediction_model

# the transformed_answer function give us a str final answer for the app
# to show in our UI
def transformed_answer(model_answer):
    if model_answer > 0.50:
        answer_transformed = "legitimate"
    else:
        answer_transformed = "fake"
    return answer_transformed

# clean_textboxes clean the textboxes from the app form
def clean_textboxes():
    return "", ""

# input_validation function check that the data required for analizis is complete
def input_validation(title_news, text_news):
    if title_news.isnumeric() or text_news.isnumeric() or text_news == "" or title_news == "":
        return 0
    else:
        return 1
# build the UI
with gr.Blocks() as FND_app:
    app_title = "Fake News Detector"
    description_a = "This app use a ML model to detect fake news"
    description_b = "Please fill the form and click **Analize** "
    gr.Markdown(description_a, show_label=False)
    gr.Markdown(description_b, show_label=False)
    gr.Label(app_title, show_label=False)
    tittle_news = gr.Textbox(placeholder="News Title here...", label="News Title")
    body_news = gr.Textbox(
        placeholder="News Text here...", label="News Text", max_lines=10
    )
    # output = gr.Textbox(label="Output Box")
    output = gr.Label(label="Answer", show_label=False)
    with gr.Row():
        greet_btn = gr.Button("Analyze")
        clean_btn = gr.Button("Clear")

    greet_btn.click(
        fn=news_analyzer,
        inputs=[tittle_news, body_news],
        outputs=output,
        api_name="FND_API",
    )

    clean_btn.click(
        fn=clean_textboxes,
        outputs=[tittle_news, body_news],
        api_name="FND_API",
    )

FND_app.launch()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e50e68e299cb320d29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [40]:
# # #test with the UI functions deactivated
# news_tittle ="***********************************************************************Shares fall as benchmark books strongest annual gain in two years"
# news_body = "The local benchmark sank in its final session of the year, as investors looked to lock in some profits to cap off a solid December run. Australian shares sank on the final trading day of 2023 on Friday, but closed the year up 7.8 per cent — the best annual performance since 2021."
# respuesta = news_analyzer(news_tittle, news_body)
# print(respuesta)

1/1 [==============================] - 0s 35ms/step
The news is fake 
